In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [5]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

In [12]:
cols = ['PONDERA_new', 'AGLOMERADO', 'CH06']

df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, usecols=cols)


In [18]:
print(map_aglomerado_region)

{'Gran La Plata': 'Pampeana', 'Bahía Blanca - Cerri': 'Pampeana', 'Gran Rosario': 'Pampeana', 'Gran Santa Fé': 'Pampeana', 'Gran Paraná': 'Pampeana', 'Posadas': 'Noreste', 'Gran Resistencia': 'Noreste', 'Comodoro Rivadavia - Rada Tilly': 'Patagonia', 'Gran Mendoza': 'Cuyo', 'Corrientes': 'Noreste', 'Gran Córdoba': 'Pampeana', 'Concordia': 'Pampeana', 'Formosa': 'Noreste', 'Neuquén – Plottier': 'Patagonia', 'Santiago del Estero - La Banda': 'Noroeste', 'Jujuy-Palpalá': 'Noroeste', 'Río Gallegos': 'Patagonia', 'Gran Catamarca': 'Noroeste', 'Gran Salta': 'Noroeste', 'La Rioja': 'Noroeste', 'Gran San Luis': 'Cuyo', 'Gran San Juan': 'Cuyo', 'Gran Tucumán - Tafí Viejo': 'Noroeste', 'Santa Rosa – Toay': 'Pampeana', 'Ushuaia - Río Grande': 'Patagonia', 'Ciudad Autónoma de Buenos Aires': 'Gran Buenos Aires', 'Partidos del GBA': 'Gran Buenos Aires', 'Mar del Plata': 'Pampeana', 'Río Cuarto': 'Pampeana', 'San Nicolás – Villa Constitución': 'Pampeana', 'Rawson – Trelew': 'Patagonia', 'Viedma – Car

In [41]:
df_temp = df_people_pool[df_people_pool['CH06']>14]
df_pob = df_temp.groupby(['AGLOMERADO']).sum(['PONDERA_new'])
df_pob.index = df_pob.index.map(dict_cod_aglomerado)
df_pob['PROVINCIA'] = df_pob.index.map(map_aglomerado_provincia)
pob_pba = df_pob[df_pob['PROVINCIA']=='Buenos Aires']['PONDERA_new'].sum()
gba = df_pob[df_pob['PROVINCIA']=='Buenos Aires'].loc['Partidos del GBA', 'PONDERA_new']
resto_pba = df_pob[df_pob['PROVINCIA']=='Buenos Aires'].loc[['Gran La Plata', 'Bahía Blanca - Cerri', 'Mar del Plata', 'Resto Buenos Aires'], 'PONDERA_new'].sum()
coef_gba = gba/pob_pba
coef_resto_pba = resto_pba/pob_pba
print(coef_gba, coef_resto_pba)

0.7453408696034303 0.25465913039656984


In [42]:
dict_regiones_ENUT = {1: 'Gran Buenos Aires',
                    2: 'Pampeana',
                    3: 'Noroeste',
                    4: 'Noreste',
                    5: 'Cuyo',
                    6: 'Patagonia'}

In [43]:
pathfolder_ENUT =  pathdata + 'data_input/ENUT_2021/Bases de datos/'
filename = 'enut2021_base.txt'
df_enut_base = pd.read_csv(pathfolder_ENUT + filename, sep='|')
columnas = pd.DataFrame(df_enut_base.columns)

In [53]:
pondera = 'WPER'
df_temp = df_enut_base[df_enut_base['EDAD_SEL']>=14]
df_selected = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())][['TSS_GRANGRUPO_TNR', 'SEXO_SEL', 'REGION', pondera]]
df_selected['Multiplication'] = df_selected['TSS_GRANGRUPO_TNR'] * df_selected[pondera]

numerador = df_selected.groupby(['SEXO_SEL', 'REGION'])['Multiplication'].sum().unstack(level=0)
denominador = df_selected.groupby(['SEXO_SEL', 'REGION'])[pondera].sum().unstack(level=0)
tasa = numerador.div(denominador, fill_value=np.nan)    
tasa = tasa.rename(columns= {2:'Varon', 1:'Mujer'})

size = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL']).size().unstack(level=1)
size = size.rename(columns= {2:'N_v', 1:'N_m'})

n_pob = df_temp[(~df_temp['TSS_GRANGRUPO_TNR'].isna())].groupby(['REGION', 'SEXO_SEL'])[pondera].sum().unstack(level=1)
n_pob = n_pob.rename(columns= {2:'N_pob_v', 1:'N_pob_m'})

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m'] 

ratio = tasa['Varon'] / tasa['Mujer']

error = pd.DataFrame(index=size.index, columns=['LI', 'LS', 'ME', 'ER'])

ratio_minutos_promedio_no_pago = pd.concat([tasa, size, ratio*100, error], axis=1)
ratio_minutos_promedio_no_pago.rename(columns={0: 'Tiempo TNR'}, inplace=True)

ratio_minutos_promedio_no_pago.index = ratio_minutos_promedio_no_pago.index.map(dict_regiones_ENUT)

In [54]:
ratio_minutos_promedio_no_pago

,Mujer,Varon,N_m,N_v,N_pob_v,N_pob_m,N_pob_tot,Tiempo TNR,LI,LS,ME,ER
REGION,,,,,,,,,,,,
Gran Buenos Aires,272.679498,137.045306,1175,993,5846495,6323252,12169747,50.258749,NaN,NaN,NaN,NaN
Pampeana,254.803949,144.061725,2617,2008,2428487,2687805,5116292,56.538262,NaN,NaN,NaN,NaN
Noroeste,256.424244,103.365229,1397,1104,1032035,1136737,2168772,40.310240,NaN,NaN,NaN,NaN
Noreste,242.804667,116.366476,746,546,536216,598613,1134829,47.925964,NaN,NaN,NaN,NaN
Cuyo,281.043165,133.301909,935,701,779446,853269,1632715,47.431116,NaN,NaN,NaN,NaN
Patagonia,245.979699,133.393950,1185,943,458211,465090,923301,54.229658,NaN,NaN,NaN,NaN


In [47]:
provincia_df = pd.DataFrame(map_provincia_region)


ValueError: If using all scalar values, you must pass an index

In [45]:
ratio_minutos_promedio_no_pago

,Mujer,Varon,N_m,N_v,N_pob_v,N_pob_m,N_pob_tot,Tiempo TNR,LI,LS,ME,ER
REGION,,,,,,,,,,,,
Gran Buenos Aires,272.679498,137.045306,1175,993,5846495,6323252,12169747,50.258749,NaN,NaN,NaN,NaN
Pampeana,254.803949,144.061725,2617,2008,2428487,2687805,5116292,56.538262,NaN,NaN,NaN,NaN
Noroeste,256.424244,103.365229,1397,1104,1032035,1136737,2168772,40.310240,NaN,NaN,NaN,NaN
Noreste,242.804667,116.366476,746,546,536216,598613,1134829,47.925964,NaN,NaN,NaN,NaN
Cuyo,281.043165,133.301909,935,701,779446,853269,1632715,47.431116,NaN,NaN,NaN,NaN
Patagonia,245.979699,133.393950,1185,943,458211,465090,923301,54.229658,NaN,NaN,NaN,NaN


In [80]:
ratio_minutos_promedio_no_pago.to_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv')